# **Video Transcription Search with OpenAI and Astra Vector Search**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

This notebook demonstrates how to use vector Search Document API in Astra DB, for storing, and computing similarity search across various types of data, including text, numercial values, images and embeddings. In this specific example we are going to store embeddings (compact representations of text as vectors of floating-point numbers) of Youtube video transcriptions, and show a couple of examples on how to retrieve them.

## **Prerequisites**
* Make sure you have an Astra DB instance and get ready to supply the corresponding Token and the API Endpoint (read more [here](https://docs.datastax.com/en/astra/astra-db-vector/)).



## Setup

### Import needed libraries

In [2]:
!pip install --quiet --upgrade astrapy datasets "openai==0.27.7" --no-deps

In [3]:
import os, json
from getpass import getpass

from astrapy.db import AstraDB

### Provide database credentials

These are the connection parameters on your Astra dashboard. Example values:

- API Endpoint: `https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com`
- Token: `AstraCS:6gBhNmsk135...`


In [4]:
ASTRA_DB_API_ENDPOINT = input("ASTRA_DB_API_ENDPOINT = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

In [5]:
# Input your database keyspace name:
ASTRA_COLLECTION_NAME = input('Your Astra DB Collection name: ')

In [6]:
astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [7]:
collection = astra_db.create_collection(ASTRA_COLLECTION_NAME, dimension=1536)

## **Dataset Setup**
We will download a public dataset of youtube video transcriptions

In [8]:
from datasets import load_dataset
dataset = load_dataset("jdabello/yt_transcriptions", split="train")
dataset

/Users/juan.abello/.pyenv/versions/3.8.16/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 384.16it/s]
Generating train split: 11298 examples [00:00, 620089.59 examples/s]


Dataset({
    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
    num_rows: 11298
})

## **Create Vector Embeddings**
In this section we are going to specify authentication credentials for the OpenAI. This will be used for generating the embeddings for our video transcriptions, as well as for the queries we are going to run.

In [9]:
apiSecret = getpass(f'Your secret for OpenAI provider: ')

In [10]:
import openai
import tqdm

# Set your OpenAI API key
openai.api_key = apiSecret
batch_size=1000

videos_list=dataset.to_list()
# Process videos in batches of 1000
for batch_start in tqdm.tqdm(range(0, len(videos_list), batch_size), desc="Processing videos"):
    batch_videos = videos_list[batch_start:batch_start + batch_size]

    # Create embeddings for the batch of videos
    embeddings = openai.Embedding.create(input=[video["text"] for video in batch_videos], model="text-embedding-ada-002")

    # Add embeddings to corresponding videos
    for i, video in enumerate(batch_videos):
      video["$vector"] = embeddings["data"][i]["embedding"]

Processing videos: 100%|██████████| 12/12 [00:28<00:00,  2.38s/it]


## Insert Videos

In [11]:
batch_size=10
for batch_start in tqdm.tqdm(range(0, len(videos_list), batch_size), desc="Ingesting videos"):
  batch_videos = videos_list[batch_start:batch_start + batch_size]
  response = collection.insert_many(batch_videos)

Ingesting videos: 100%|██████████| 1130/1130 [04:06<00:00,  4.59it/s]


## Find documents

Find by `_id`:

In [12]:
document = collection.find_one(filter={"_id":"4eaf67d0-d612-4ab5-b096-7169b6001363"})
print(document)

{'data': {'document': None}}


Find by any (non-vector) filter clause:

In [13]:
document = collection.find_one(filter={"title":"Predictive Coding Models of Perception"})
print(document)


{'data': {'document': {'_id': '151ac7b7-2713-4c75-9936-e8890268918a', 'video_id': 'P0yVuoATjzs', 'text': " we're gonna show it these stimuli that are basically the same as what you would find in these experiments in an untrained pred net you get these sort of funny effects where it's you know you flash up an image and it takes a little while to sort of predict it away but then as you train these these sequences so that are expected you get sharper and sharper transitions from between these", 'start_second': 1633, 'end_second': 1654, 'url': 'https://www.youtube.com/watch?v=P0yVuoATjzs&t=1633s', 'title': 'Predictive Coding Models of Perception', 'thumbnail': 'https://i.ytimg.com/vi/P0yVuoATjzs/maxresdefault.jpg', '$vector': [-0.020195063203573227, -0.013157390058040619, 0.020589444786310196, 0.009587555192410946, 0.008873588405549526, 0.010627907700836658, 0.003083657007664442, -0.025621213018894196, -0.016904016956686974, -0.00971674919128418, 0.0030122604221105576, 0.02989141456782818,

### Find by vector similarity

By default, the `$similarity` field is returned with each document (note the decreasing order):

In [14]:
query_vector = [-0.018835393711924553,0.03362218290567398,0.007775465026497841,-0.04818107187747955,-0.021449558436870575,0.02592715434730053,0.0025907044764608145,-0.01707921177148819,-0.018513649702072144,-0.014210332185029984,0.027415217831730843,0.03662512078881264,-0.014290768653154373,-0.013600360602140427,-0.0016020139446482062,0.014585699886083603,0.02482786402106285,0.021422747522592545,-0.009960638359189034,-0.019358227029442787,-0.013312132097780704,-0.013560143299400806,-0.01233349647372961,-0.009598677046597004,0.010523689910769463,-0.0007482209475710988,-0.004534570965915918,-0.03271057829260826,-0.03485553339123726,-0.0035458800848573446,-0.007654811255633831,0.004199421498924494,-0.005245087202638388,-0.02450612001121044,0.010818621143698692,-0.0034855531994253397,-0.01836618408560753,-0.007105166092514992,0.019438661634922028,-0.01047676894813776,0.027696741744875908,0.019036483019590378,-0.024385467171669006,0.005637212190777063,0.006478437222540379,-0.014129896648228168,-0.011884395964443684,-0.019331414252519608,0.007185602094978094,0.009317152202129364,0.022575659677386284,0.019894465804100037,-0.030753305181860924,0.010416441597044468,0.014934254810214043,-0.0030867254827171564,0.006649363320320845,-0.0005969848134554923,0.014625918120145798,-0.00886134896427393,-0.018687928095459938,0.022039420902729034,-0.026181867346167564,0.014692947268486023,-0.0030783468391746283,0.0004103484970983118,0.014384610578417778,0.018647709861397743,0.03456060215830803,0.006166748236864805,0.015430276282131672,0.0036799400113523006,-0.021851737052202225,0.00001810592038964387,0.02108759805560112,-0.010832027532160282,-0.03469466045498848,-0.028688784688711166,-0.01632847636938095,0.008331812918186188,-0.001928784535266459,-0.020618388429284096,-0.028608348220586777,0.028742408379912376,0.003669885452836752,0.02154340036213398,0.03386349231004715,0.021637242287397385,0.00908925011754036,-0.010657749138772488,0.03338087722659111,0.02514960803091526,0.027509059756994247,0.022548848763108253,-0.0067968289367854595,-0.004926695488393307,-0.025752875953912735,0.021744489669799805,-0.036973677575588226,-0.025511568412184715,-0.0005433608894236386,0.02119484543800354,-0.009853390976786613,-0.019250979647040367,-0.04091503471136093,-0.01309763640165329,0.025672441348433495,-0.013995837420225143,0.019398445263504982,-0.04188026487827301,-0.02122165635228157,0.039440374821424484,-0.004330129828304052,-0.04043241962790489,-0.006143287755548954,0.010738185606896877,0.004628412425518036,-0.003961465321481228,-0.029412707313895226,-0.008573120459914207,0.013499815948307514,-0.0001628616446396336,0.0245597455650568,-0.008774209767580032,0.026744918897747993,0.02264269068837166,-0.0228571854531765,-0.00916968658566475,0.0054260678589344025,0.011777148582041264,0.0038642720319330692,0.0205781701952219,-0.006950997747480869,0.03761716187000275,-0.023178929463028908,0.01584586128592491,0.015403464436531067,0.023178929463028908,0.0179371926933527,-0.02017599157989025,0.0004930885042995214,0.018325965851545334,-0.011609573848545551,0.017548419535160065,-0.0169987753033638,0.06113124266266823,0.016904933378100395,0.000371177913621068,-0.007996663451194763,-0.00839214026927948,0.0010297463741153479,-0.01646253652870655,0.02173108421266079,-0.02199920453131199,-0.0058617619797587395,0.03649106249213219,-0.004105579573661089,0.025297073647379875,-0.019291196018457413,0.013834965415298939,-0.00022601634555030614,0.0019488935358822346,0.03590120002627373,0.013412676751613617,0.028313416987657547,0.024090535938739777,0.025940559804439545,-0.013439489528536797,-0.007627999410033226,-0.010557204484939575,-0.028072109445929527,0.027254344895482063,0.024760834872722626,0.01982743665575981,-0.025618815794587135,0.011113552376627922,0.02469380386173725,-0.01772269792854786,-0.012420634739100933,-0.023809010162949562,-0.015912890434265137,0.0017779673216864467,-0.001766237081028521,-0.01329872664064169,-0.006458328105509281,0.00065479805925861,0.02162383683025837,-0.011850881390273571,0.01866111531853676,-0.002748224651440978,0.02665107697248459,0.015698395669460297,-0.011488920077681541,-0.016583189368247986,-0.6168892979621887,-0.022656096145510674,-0.00006420204590540379,-0.015229186974465847,-0.024278219789266586,0.016610002145171165,-0.016315070912241936,-0.011408483609557152,-0.03316637873649597,0.02242819406092167,-0.01694515161216259,0.01638210006058216,0.016368694603443146,-0.02458655647933483,-0.04078097268939018,-0.018218718469142914,-0.020015118643641472,-0.00886134896427393,0.003911192994564772,-0.013794747181236744,-0.011401780880987644,0.0033163027837872505,-0.011549246497452259,0.013271914795041084,0.008224565535783768,0.004320075269788504,0.013506518676877022,-0.003686642972752452,-0.002049438189715147,0.024251407012343407,-0.030404750257730484,0.013466301374137402,-0.003968168515712023,-0.0321207158267498,0.04011067375540733,0.019867653027176857,-0.008733992464840412,0.016020139679312706,0.01780313439667225,0.002738170325756073,-0.03217433765530586,-0.018982859328389168,0.012889844365417957,0.013070824556052685,-0.025055766105651855,0.027964862063527107,-0.005553424824029207,-0.010121510364115238,0.014786789193749428,-0.013955619186162949,0.008130723610520363,0.005523261148482561,-0.03150403872132301,0.0001797238364815712,0.02485467679798603,0.006984512321650982,0.024573151022195816,-0.01022205501794815,0.005442825611680746,0.01252118032425642,0.006870561745017767,0.005124433431774378,-0.023044869303703308,-0.01274908147752285,-0.04477595537900925,0.012796002440154552,-0.0204441100358963,-0.006327619776129723,0.0025890287943184376,-0.0030783468391746283,0.005007131490856409,0.015242592431604862,-0.002423129742965102,-0.0016397181898355484,-0.027535870671272278,0.04263099655508995,0.0072325230576097965,-0.003763727145269513,-0.028661971911787987,0.029761262238025665,0.01791038177907467,0.0013355701230466366,-0.028259793296456337,-0.02600758895277977,0.03466784954071045,-0.018580680713057518,-0.011328048072755337,0.019974902272224426,0.0014964418951421976,-0.005402607377618551,0.005174706224352121,-0.003498959355056286,-0.011797257699072361,-0.029064152389764786,-0.018580680713057518,0.010175134055316448,0.005121082067489624,0.0004679522826336324,0.00016673680511303246,0.00007383758929790929,-0.009230013005435467,0.009886905550956726,0.04021792113780975,0.005405959207564592,0.010134916752576828,-0.003994980361312628,-0.019961494952440262,0.01090576034039259,0.0031001316383481026,-0.04163895547389984,0.005791381001472473,-0.01124090887606144,-0.0007616269285790622,0.011770444922149181,0.0011537516256794333,-0.03367580845952034,0.022776750847697258,0.011113552376627922,-0.014250550419092178,-0.02710687927901745,0.019653158262372017,-0.007493939716368914,0.023661544546484947,-0.010557204484939575,0.023205740377306938,0.047564394772052765,-0.011006304994225502,-0.01648934744298458,-0.008043584413826466,0.013137854635715485,-0.017307111993432045,-0.016046950593590736,0.010604125447571278,-0.03453379124403,0.0011420213850215077,0.003901138436049223,0.011180582456290722,-0.02049773372709751,0.00887475535273552,-0.038421522825956345,-0.015095126815140247,-0.015805643051862717,0.011837475001811981,-0.016556378453969955,-0.013533330522477627,-0.01053039263933897,-0.01860749162733555,0.0024449145421385765,-0.011542543768882751,-0.04072735086083412,-0.0022555552423000336,-0.01333894394338131,-0.00874069519340992,0.0008764155791141093,0.010691264644265175,-0.018781770020723343,0.012534585781395435,-0.037965718656778336,-0.004541273694485426,-0.01035611517727375,-0.007219117134809494,0.02547135017812252,-0.0007511535077355802,0.02052454650402069,-0.010744888335466385,-0.009149577468633652,-0.01047676894813776,0.0014076272491365671,0.0033984144683927298,-0.007956445217132568,-0.014438234269618988,-0.021690865978598595,-0.013211587443947792,0.02092672511935234,-0.0012961900793015957,0.00440386263653636,-0.006093015428632498,0.020618388429284096,-0.0018165095243602991,-0.01654297299683094,-0.013412676751613617,0.0019589478615671396,0.0032124065328389406,-0.0021801465190947056,0.01640891283750534,0.013479706831276417,0.02060498297214508,0.03761716187000275,-0.006451624911278486,0.039922989904880524,0.0025538380723446608,-0.003192297648638487,-0.030217066407203674,0.004531219135969877,-0.013211587443947792,-0.012313387356698513,0.01191791146993637,-0.008821130730211735,0.001374112325720489,0.047993388026952744,0.004702145233750343,-0.00667617516592145,0.027415217831730843,-0.03386349231004715,-0.004685387946665287,-0.026986226439476013,0.006599090993404388,-0.03289826214313507,-0.007567672524601221,0.023192334920167923,-0.002783415373414755,-0.004356941673904657,-0.011683306656777859,-0.0022203645203262568,0.008036881685256958,-0.008050287142395973,-0.008827834390103817,0.030002569779753685,-0.02073904126882553,0.006052797194570303,-0.009886905550956726,-0.008472575806081295,0.014210332185029984,-0.003081698203459382,-0.0013975728070363402,0.01581905037164688,0.018165094777941704,0.03477509692311287,0.006863858550786972,-0.01285632885992527,0.006022633984684944,0.008305001072585583,0.00016820308519527316,0.02423800155520439,0.01777632161974907,0.0008671989780850708,0.004628412425518036,-0.008036881685256958,0.026423174887895584,0.016261447221040726,0.008941784501075745,0.023674950003623962,0.004078767728060484,-0.007333067711442709,0.018875611945986748,0.0169987753033638,0.03195984289050102,-0.014330985955893993,-0.02098034881055355,-0.011656494811177254,-0.023312989622354507,0.004377050790935755,-0.002250527963042259,-0.012534585781395435,0.0072928499430418015,-0.021529994904994965,-0.01799081824719906,0.015671582892537117,0.038555581122636795,0.029922135174274445,-0.005070809740573168,-0.00275660352781415,-0.006837046705186367,-0.008345219306647778,0.00043192371958866715,0.005436122417449951,-0.02445249632000923,-0.01705239899456501,-0.037697598338127136,-0.0001859031617641449,-0.030458373948931694,-0.0019271087367087603,0.01635528914630413,-0.02388944663107395,0.018299154937267303,0.019559316337108612,-0.0007876009913161397,-0.00010787619976326823,0.006237129680812359,0.03429248183965683,-0.019720187410712242,-0.0054126619361341,0.021784707903862,-0.0009669059072621167,0.019907871261239052,-0.007165492977946997,-0.013901995494961739,0.007312959060072899,-0.02089991420507431,0.006485139951109886,-0.004976967815309763,0.02004193142056465,-0.02122165635228157,-0.007111869286745787,-0.01030249148607254,-0.001194807467982173,0.024948518723249435,-0.009008814580738544,0.021905362606048584,-0.0011755363084375858,-0.0034285779111087322,0.011723523959517479,0.014786789193749428,-0.007038136478513479,0.05694857984781265,0.02480105310678482,0.00655552139505744,-0.018004223704338074,0.006622551009058952,-0.021020567044615746,-0.0035190682392567396,0.01215921901166439,-0.0031872703693807125,-0.038743264973163605,-0.0003081279282923788,0.009746143594384193,0.007393394596874714,0.019679969176650047,0.037858471274375916,0.011006304994225502,-0.007326364982873201,-0.006806883495301008,-0.024599961936473846,-0.009960638359189034,0.04051285237073898,0.047644831240177155,-0.0027281157672405243,-0.002977801952511072,0.0014503587735816836,-0.007426909636706114,-0.01654297299683094,-0.005429419688880444,0.037992529571056366,-0.019921276718378067,-0.004159203264862299,0.01071807648986578,-0.006837046705186367,-0.019840842112898827,0.007902821525931358,0.01071807648986578,-0.016931746155023575,-0.008117317222058773,-0.011777148582041264,-0.0133523503318429,0.003669885452836752,-0.0017142889555543661,0.02611483819782734,0.031101860105991364,0.027080068364739418,0.00837873388081789,0.009002111852169037,0.02630252204835415,-0.0006468382780440152,-0.03994980454444885,0.0002004612033488229,0.024157565087080002,0.014250550419092178,0.01855386793613434,-0.03563307970762253,-0.0018751606112346053,0.003279436379671097,0.0029426112305372953,-0.022870592772960663,0.019197354093194008,-0.022977840155363083,0.008103911764919758,0.0254981629550457,-0.019961494952440262,-0.006930888630449772,-0.02547135017812252,-0.00815083272755146,0.0019522450165823102,-0.02264269068837166,-0.013023903593420982,0.020591575652360916,-0.008767507039010525,-0.03423885628581047,-0.009638895280659199,0.017977410927414894,-0.021798113361001015,-0.002934232586994767,-0.017521608620882034,0.015309622511267662,-0.00017197351553477347,0.008318406529724598,-0.01276248786598444,0.015282810665667057,-0.000627567176707089,-0.023393424227833748,-0.0038542174734175205,0.021449558436870575,0.012916656211018562,-0.014987879432737827,-0.007145384326577187,-0.01860749162733555,-0.0005680781323462725,-0.010248866863548756,0.01796400547027588,0.03263014182448387,0.017655668780207634,0.004718902986496687,-0.009531647898256779,0.0043502384796738625,0.03600844740867615,-0.013191478326916695,-0.029573578387498856,0.001053206855431199,0.0004185177676845342,-0.014518669806420803,0.011938019655644894,-0.004635115619748831,-0.0205781701952219,0.014706353656947613,-0.002639301121234894,0.023715168237686157,-0.00034310916089452803,0.00015542551409453154,0.010637640953063965,0.0010649370960891247,0.022012609988451004,0.019800623878836632,0.021127814427018166,0.02039048634469509,-0.04863687604665756,-0.00039149634540081024,-0.0023343153297901154,-0.01758863776922226,-0.0017377494368702173,-0.009022220969200134,-0.020403891801834106,-0.0011235881829634309,0.00013657336239703,-0.004635115619748831,-0.0019505692180246115,0.02103397436439991,-0.0033682510256767273,0.02611483819782734,-0.011126958765089512,0.013653984293341637,0.015752019360661507,0.004893180448561907,0.020993756130337715,0.008372031152248383,-0.017414361238479614,0.002141604432836175,-0.03380986675620079,0.03190621733665466,-0.018929235637187958,-0.04011067375540733,0.006394649855792522,0.02490830048918724,-0.010637640953063965,-0.03190621733665466,0.012239654548466206,-0.020243020728230476,0.030270690098404884,-0.02189195528626442,-0.02261587791144848,-0.019385037943720818,-0.006930888630449772,-0.020511141046881676,0.03343449905514717,-0.00457814009860158,-0.0004461675707716495,-0.015510711818933487,-0.011803960427641869,0.0038709749933332205,-0.010791809298098087,0.007433612830936909,-0.05075501650571823,-0.000039615701098227873,0.03469466045498848,0.005657321307808161,0.02689238451421261,-0.0030515347607433796,0.0038475145120173693,-0.011415187269449234,-0.015001284889876842,0.01640891283750534,-0.05123763158917427,-0.017467984929680824,0.014089678414165974,0.03418523445725441,0.03244245797395706,0.0018416456878185272,-0.00839214026927948,0.010744888335466385,0.009303745813667774,0.004531219135969877,0.0028973661828786135,-0.0073196617886424065,0.021315498277544975,-0.018084658309817314,0.0167306549847126,0.0015802292618900537,-0.0028789329808205366,0.018272342160344124,-0.010697967372834682,0.028581537306308746,0.03351493552327156,0.009913718327879906,-0.026423174887895584,-0.005831598769873381,-0.0014411421725526452,-0.025726065039634705,0.025109389796853065,-0.002612489275634289,-0.01280270516872406,-0.03128954395651817,0.018781770020723343,-0.022763343527913094,0.007983257994055748,-0.007762059103697538,0.007339770905673504,0.03201346471905708,-0.020216209813952446,0.01732051931321621,-0.022709719836711884,0.013540034182369709,-0.024117346853017807,-0.021449558436870575,-0.019800623878836632,-0.0020427352283149958,0.010134916752576828,0.0038877325132489204,0.01812487654387951,0.01124090887606144,0.018004223704338074,0.0011721848277375102,0.023996694013476372,-0.010503580793738365,0.017387548461556435,-0.005174706224352121,-0.03263014182448387,0.00042291657882742584,0.0064415703527629375,-0.03284463658928871,0.010382927022874355,-0.009752846322953701,0.037643976509571075,-0.01142859272658825,0.01841980777680874,-0.024251407012343407,-0.015725208446383476,0.03421204537153244,0.017602043226361275,0.03332725167274475,0.004551328253000975,0.027562683448195457,0.01113366149365902,0.00619356008246541,0.0030682922806590796,-0.01635528914630413,-0.00899540912359953,0.004511110484600067,0.012809407897293568,0.014089678414165974,-0.029573578387498856,-0.010295787826180458,0.01662340760231018,0.019907871261239052,-0.004504407290369272,-0.022816967219114304,0.015188968740403652,0.008472575806081295,0.020725635811686516,-0.0007905335514806211,-0.025109389796853065,-0.03509683907032013,0.011870990507304668,-0.015711801126599312,0.002781739691272378,-0.005989118944853544,-0.014505264349281788,-0.011502325534820557,0.047912951558828354,-0.01035611517727375,0.03402436152100563,0.0003305410500615835,-0.010811918415129185,-0.004501055926084518,0.004330129828304052,-0.007493939716368914,0.00803017895668745,0.0030414804350584745,0.018969453871250153,-0.012688754126429558,-0.0009049032814800739,0.02240138314664364,0.021503182128071785,-0.016127387061715126,-0.015752019360661507,0.009323854930698872,0.03622294217348099,-0.02498873509466648,-0.003328033024445176,-0.011669900268316269,-0.016931746155023575,0.011468810960650444,-0.03179896995425224,0.006344377063214779,-0.023071682080626488,-0.019183948636054993,-0.011019710451364517,0.04488320276141167,0.021422747522592545,0.001963975140824914,0.007514048367738724,-0.007902821525931358,-0.011576058343052864,-0.017119428142905235,0.021637242287397385,-0.006103069521486759,-0.041424460709095,-0.03190621733665466,-0.027643118053674698,0.0016162577085196972,0.021127814427018166,0.0009627165272831917,-0.01563136652112007,-0.007312959060072899,0.02716050297021866,0.006776719819754362,0.022763343527913094,0.004470892250537872,0.002798497211188078,-0.03222796320915222,-0.005664024036377668,0.001007123850286007,-0.027857614681124687,-0.022495225071907043,-0.02557859942317009,-0.025940559804439545,-0.029225023463368416,0.03187940642237663,0.04134402424097061,0.00892837904393673,0.0029308809898793697,0.020082149654626846,0.01600673235952854,0.003797242185100913,0.0027163855265825987,-0.05268547683954239,0.013821559026837349,-0.007219117134809494,-0.010858839377760887,0.014840413816273212,-0.014009242877364159,-0.01732051931321621,0.008713883347809315,0.008291594684123993,0.005798083730041981,-0.021771302446722984,-0.003569340566173196,-0.00314034940674901,0.013050715439021587,-0.013687499798834324,0.01225306000560522,0.014129896648228168,-0.015336434356868267,-0.022776750847697258,-0.0019740296993404627,0.0038005937822163105,-0.029037339612841606,0.013781341724097729,0.01030249148607254,0.008827834390103817,0.016261447221040726,-0.023728573694825172,0.0025873531121760607,-0.007802276872098446,-0.027535870671272278,-0.055447109043598175,-0.006924185436218977,-0.019800623878836632,0.04171939194202423,0.017977410927414894,-0.014947661198675632,-0.006806883495301008,-0.011609573848545551,-0.04209475964307785,-0.009390885010361671,-0.011830772273242474,0.00851279404014349,0.029814885929226875,0.007500642444938421,-0.038528770208358765,0.012474259361624718,0.012997091747820377,0.028715597465634346,-0.03659830987453461,-0.016422318294644356,0.015162156894803047,0.023366613313555717,0.015564336441457272,0.01742776669561863,-0.02673151157796383,-0.00794974248856306,0.04507088661193848,-0.014934254810214043,-0.008894863538444042,0.0098198764026165,-0.019304603338241577,0.0017528311582282186,0.010322599671781063,0.017038993537425995,0.015443682670593262,0.009055735543370247,0.030056193470954895,-0.030002569779753685,0.008539605885744095,0.0023611271753907204,-0.0023661544546484947,-0.011100146919488907,-0.01299038901925087,0.0005366579280234873,0.024385467171669006,0.014103084802627563,-0.017896976321935654,-0.014036054722964764,0.00908925011754036,0.006190208718180656,0.021958986297249794,0.02221369929611683,0.02608802542090416,0.014411422424018383,0.009028923697769642,-0.012480962090194225,-0.020162584260106087,0.005784677807241678,-0.005043997894972563,0.028045298531651497,0.006997918710112572,0.00275660352781415,0.011354859918355942,0.008385436609387398,0.014250550419092178,-0.04466870427131653,0.011736930347979069,-0.007969851605594158,-0.013003794476389885,-0.04145127162337303,0.012011752463877201,-0.007822385989129543,-0.03576713800430298,-0.004899883642792702,-0.0031252678018063307,-0.00008525361772626638,-0.0010439902544021606,-0.014210332185029984,0.019478879868984222,-0.031048236414790154,0.01603354513645172,0.003612909931689501,0.026101430878043175,-0.019250979647040367,-0.0353381484746933,0.01788356900215149,-0.01758863776922226,-0.004661927465349436,0.17524290084838867,-0.010731481947004795,-0.018580680713057518,0.02667788788676262,0.003388359909877181,-0.010691264644265175,0.009189795702695847,0.0007687488105148077,-0.009907014667987823,0.004755769390612841,0.008305001072585583,0.00016537525516469032,-0.02135571651160717,-0.00418266374617815,0.017843350768089294,-0.006240481045097113,-0.008010069839656353,0.005821544211357832,-0.012132406234741211,-0.01250777393579483,0.010617531836032867,-0.0025471351109445095,0.03418523445725441,-0.006622551009058952,0.03338087722659111,0.0036430733744055033,0.010255570523440838,-0.03466784954071045,-0.0006300807581283152,0.019183948636054993,-0.03311275690793991,-0.0036062069702893496,0.009491429664194584,-0.0017067480366677046,-0.011690009385347366,0.00430331751704216,0.005033943336457014,-0.005164651665836573,0.030753305181860924,0.027696741744875908,0.037858471274375916,-0.00047758783330209553,0.013982431031763554,-0.0033414389472454786,-0.02264269068837166,0.0036564795300364494,-0.006029336713254452,-0.005563479382544756,0.005157948471605778,-0.008264782838523388,-0.05984427034854889,-0.005731054116040468,0.024546338245272636,0.016181010752916336,0.01777632161974907,0.0068839676678180695,0.021503182128071785,0.018808580935001373,0.013493113219738007,0.027696741744875908,-0.012708863243460655,0.013928807340562344,-0.008103911764919758,0.04421290382742882,0.011073335073888302,0.0008814428001642227,0.008050287142395973,0.007038136478513479,0.020537951961159706,-0.013781341724097729,0.011475513689219952,-0.014438234269618988,0.00023418561613652855,-0.007038136478513479,-0.014196926727890968,0.017870163545012474,0.05225648730993271,0.006203614640980959,0.008103911764919758,-0.01047676894813776,0.0026677888818085194,0.014518669806420803,-0.011193988844752312,0.019331414252519608,-0.028125733137130737,-0.017521608620882034,0.04075416177511215,-0.007762059103697538,-0.0001213869036291726,-0.02581990696489811,0.003911192994564772,0.025243449956178665,-0.012661942280828953,-0.004045252688229084,0.010798512026667595,-0.020484328269958496,-0.005322171840816736,0.0017896975623443723,-0.02207963913679123,-0.009498132392764091,-0.0021818222012370825,0.046947721391916275,0.008519496768712997,0.019318008795380592,-0.005486394744366407,-0.005888574291020632,0.006297456566244364,0.009531647898256779,0.004427323117852211,-0.02194557897746563,-0.006639308761805296,-0.029090963304042816,0.015577741898596287,0.0014612511731684208,0.007889416068792343,0.019545910879969597,0.014062866568565369,-0.04402521997690201,0.022334352135658264,-0.00910265650600195,-0.010852135717868805,-0.005556776188313961,-0.008103911764919758,0.027321375906467438,0.007882712408900261,-0.0002880189858842641,0.0024298327043652534,-0.001912027015350759,0.019706781953573227,-0.020886506885290146,0.03574032709002495,-0.0073464736342430115,0.017253488302230835,-0.016904933378100395,-0.007936337031424046,-0.0035123652778565884,0.005466286092996597,0.004417268559336662,-0.01568499021232128,0.007433612830936909,-0.017494795843958855,0.011743633076548576,0.00832511018961668,0.000560956250410527,-0.0019053240539506078,-0.00676331389695406,0.014223738573491573,0.02167746052145958,-0.010811918415129185,0.006528709549456835,0.015993326902389526,0.004051955882459879,-0.0014587375335395336,0.011053225956857204,0.017065804451704025,-0.001599500305019319,-0.004883125890046358,-0.014465046115219593,-0.009605380706489086,0.0036229644902050495,-0.02504236064851284,0.0024868082255125046,-0.014062866568565369,-0.0011395078618079424,-0.019438661634922028,-0.007279444020241499,-0.1678427904844284,0.006904076784849167,0.03469466045498848,-0.014170114882290363,0.005851707886904478,-0.010295787826180458,0.02600758895277977,0.013667390681803226,-0.006511951796710491,0.007379988674074411,-0.007748653180897236,-0.014263956807553768,-0.009766251780092716,-0.019814029335975647,-0.018460026010870934,-0.008941784501075745,-0.009980747476220131,0.011971535161137581,0.03335406258702278,0.0009895284892991185,0.002191876759752631,-0.02606121450662613,0.01643572375178337,-0.007527454290539026,0.023071682080626488,0.031021423637866974,-0.0061064213514328,0.007627999410033226,0.0063611348159611225,-0.037483103573322296,-0.0003600760828703642,0.0004646008019335568,0.009518241509795189,0.018245531246066093,0.038126591593027115,0.013560143299400806,0.010832027532160282,0.005318820476531982,-0.01958612911403179,0.013392568565905094,0.03595482185482979,0.013888589106500149,-0.011837475001811981,0.0016816118732094765,-0.0019086755346506834,0.01597992144525051,-0.009471320547163486,0.00505070062354207,-0.029600391164422035,-0.028635160997509956,0.007627999410033226,-0.02081947773694992,-0.007507345639169216,-0.002381236059591174,-0.005007131490856409,-0.015832455828785896,0.014505264349281788,0.02028323896229267,0.014103084802627563,-0.022602472454309464,-0.018218718469142914,-0.03466784954071045,0.047859326004981995,-0.009531647898256779,-0.014411422424018383,-0.015336434356868267,-0.014920849353075027,0.007024730555713177,-0.03327362611889839,0.00556683074682951,-0.005429419688880444,-0.02036367543041706,0.009069141931831837,-0.023245958611369133,0.009216607548296452,0.010818621143698692,-0.011006304994225502,-0.006391298025846481,-0.004122336860746145,0.008358624763786793,0.024103941395878792,0.0072861467488110065,-0.025833312422037125,-0.008144129067659378,0.0027767124120146036,0.0005136163672432303,0.009846688248217106,0.024894893169403076,0.015658177435398102,0.003911192994564772,0.01293676532804966,-0.006491843145340681,-0.0014076272491365671,-0.02124846912920475,0.024251407012343407,0.008646853268146515,-0.010805214755237103,0.007467127405107021,0.005255141761153936,0.017226677387952805,-0.009303745813667774,0.01627485267817974,-0.014049461111426353,0.021771302446722984,0.027147097513079643,-0.006351080257445574,-0.03354174643754959,0.009444508701562881,0.03228158503770828,-0.0063343229703605175,-0.04348897933959961,-0.010691264644265175,-0.00440386263653636,0.0009928799699991941,-0.004534570965915918,0.0020159233827143908,0.011542543768882751,0.0033062484581023455,0.046036116778850555,0.030029382556676865,0.04324767366051674,0.011334750801324844,-0.001319650560617447,0.015282810665667057,-0.021261874586343765,-0.025189826264977455,-0.11990303546190262,-0.03354174643754959,0.014894037507474422,0.01817850023508072,-0.00400838628411293,0.02662426419556141,0.006528709549456835,0.038609206676483154,-0.014237144961953163,0.027643118053674698,-0.045473065227270126,0.005342280492186546,0.0022790157236158848,-0.0028722300194203854,-0.007152087055146694,-0.017307111993432045,-0.0053121172823011875,-0.00619356008246541,-0.015014691278338432,0.009270231239497662,-0.009250122122466564,-0.02477424032986164,-0.003492256160825491,0.02175789698958397,0.018768364563584328,-0.0037168064154684544,-0.015457088127732277,0.0035793951246887445,0.01836618408560753,-0.03324681520462036,0.005891925655305386,-0.030029382556676865,0.006133233197033405,-0.01124090887606144,-0.0006899887230247259,-0.00908925011754036,-0.008465873077511787,-0.026101430878043175,0.017736103385686874,-0.023969881236553192,-0.00908925011754036,0.024546338245272636,0.0077553559094667435,0.003361548064276576,-0.029171399772167206,-0.007118572480976582,-0.0168647151440382,0.02020280249416828,-0.014692947268486023,-0.0006108097149990499,-0.03633018955588341,-0.01847343146800995,-0.03313956782221794,-0.030726492404937744,0.03332725167274475,-0.01756182685494423,-0.0072057112120091915,-0.008700476959347725,-0.002433184301480651,-0.006371189374476671,-0.023098492994904518,0.009772955439984798,-0.00034122393117286265,-0.017414361238479614,0.007373285945504904,-0.015765424817800522,-0.019063295796513557,-0.04190707579255104,0.031343165785074234,-0.018647709861397743,0.018647709861397743,-0.004514461848884821,0.0009400939452461898,0.0253641027957201,-0.0446418933570385,-0.012889844365417957,-0.022951027378439903,-0.02014917880296707,0.011850881390273571,0.013821559026837349,-0.017709292471408844,-0.0035894496832042933,0.012098891660571098,-0.021690865978598595,0.03627656772732735,-0.0019271087367087603,0.007849197834730148,-0.024117346853017807,-0.006716392934322357,-0.027442028746008873,-0.01340597402304411,0.010094698518514633,0.011703415773808956,-0.003981574438512325,-0.023567702621221542,-0.02148977667093277,-0.01268205139786005,-0.0013079203199595213,0.022830374538898468,0.004387104883790016,-0.024331843480467796,-0.0321207158267498,-0.013271914795041084,0.011247612535953522,-0.015644771978259087,-0.021315498277544975,0.024331843480467796,-0.016100574284791946,0.019103514030575752,-0.006206966005265713,-0.004001683089882135,0.011904505081474781,-0.012299980968236923,0.02248181775212288,-0.005268547683954239,0.0031085102818906307,-0.02673151157796383,-0.008941784501075745,0.026664482429623604,-0.02186514437198639,0.04461508244276047,0.027173910290002823,0.007520751561969519,-0.01268205139786005,0.002466699341312051,0.00784249510616064,-0.038421522825956345,0.004708848427981138,-0.02234775945544243,0.010362817905843258,-0.012078782543540001,-0.03547220677137375,0.0019237572560086846,-0.03692005202174187,-0.029198212549090385,0.011817365884780884,0.005385850090533495,-0.015483899973332882,0.012836220674216747,0.027830801904201508,-0.005382498726248741,0.0499238483607769,-0.013647281564772129,-0.05142531543970108,0.004765823949128389,-0.012688754126429558,-0.001497279736213386,-0.0086334478110075,-0.05812830477952957,0.029064152389764786,0.014170114882290363,-0.010081292130053043,0.014625918120145798,0.018111471086740494,-0.015229186974465847,-0.023648139089345932,-0.0021449557971209288,-0.007775465026497841,0.010778402909636497,-0.003138673724606633,-0.01573861390352249,-0.012688754126429558,0.04236287996172905,0.004028495401144028,0.003252624534070492,-0.005912034772336483,0.00043192371958866715,0.0065387641079723835,-0.04300636425614357,0.006930888630449772,0.009136171080172062,-0.03994980454444885,-0.008224565535783768,0.010014262981712818,0.028849655762314796,-0.0027700094506144524,0.0003334736102260649,0.006113124080002308,-0.00824467372149229,-0.0036631824914366007,0.00563386082649231,0.009960638359189034,0.012702160514891148,-0.015886079519987106,-0.016904933378100395,0.012487664818763733,0.03515046462416649,-0.013372459448873997,-0.002677843440324068,0.03563307970762253,0.00812401995062828,-0.007473830599337816,0.0021902010776102543,0.018862206488847733,-0.023312989622354507,0.018781770020723343,0.018808580935001373,0.00315710692666471,0.008305001072585583,-0.005845004692673683,-0.006461679469794035,0.03228158503770828,0.012541288509964943,0.017736103385686874,0.022991245612502098,-0.026852166280150414,-0.017816539853811264,0.007815683260560036,-0.01614079251885414,-0.039413563907146454,0.010054480284452438,0.009243419393897057,-0.01732051931321621,-0.008164238184690475,-0.023286176845431328,-0.0013330565998330712,0.01581905037164688,0.023929663002490997,-0.00009886905900202692,0.0008190212538465858,-0.02229413576424122,0.014196926727890968,0.007359880022704601,-0.016797685995697975,0.012467555701732635,-0.008284891955554485,-0.002431508619338274,0.015497306361794472,0.023406831547617912,-0.02106078527867794,0.018862206488847733,-0.003165485570207238,0.013969024643301964,-0.02041729912161827,0.0012442419538274407,-0.02488148771226406,-0.012393822893500328,-0.018352778628468513,-0.011448701843619347,0.018379591405391693,-0.008827834390103817,0.06064862757921219,0.0028286606539040804,-0.018218718469142914,0.005499801132827997,-0.05306084454059601,0.0017997520044445992,0.00891497265547514,0.004829502198845148,-0.022709719836711884,-0.028983715921640396,0.029654014855623245,-0.014009242877364159,0.015550930052995682,-0.028125733137130737,-0.017333924770355225,0.0034269022289663553,0.0005345632089301944,0.029010528698563576,0.009886905550956726,0.011180582456290722,0.036678746342659,-0.010604125447571278,0.015282810665667057,0.006143287755548954,-0.011817365884780884,-0.0004679522826336324,0.03994980454444885,0.0040854704566299915,-0.01659659668803215,-0.03209390118718147,0.0016522862715646625,-0.004018440842628479,-0.038904137909412384,-0.014639323577284813,-0.014290768653154373,-0.009377478621900082,-0.007158790249377489,-0.002533729188144207,0.04922673851251602,0.030511997640132904,0.02028323896229267,0.016851309686899185,-0.013452894985675812,-0.011984940618276596,0.006015930790454149,0.009437805972993374,-0.014599106274545193,-0.028152545914053917,-0.02447930909693241]

documents = collection.vector_find(query_vector, limit=5)
for document in documents:
    print(f"\n{document}")


{'_id': '07b43be5-fa7b-4b85-abdc-821dbba4f96d', 'video_id': 'P0yVuoATjzs', 'text': " feedback synapses specifically and ask how the information that they contained is different than the the cell bodies in layer 2 3 so these are all experiments that are ongoing but I don't have a good answer for you but that's the right kind of question and that's the right kind of question we can ask you know when we have these kinds of models they can guide our experiments ok so this is my", 'start_second': 2196, 'end_second': 2214, 'url': 'https://www.youtube.com/watch?v=P0yVuoATjzs&t=2196s', 'title': 'Predictive Coding Models of Perception', 'thumbnail': 'https://i.ytimg.com/vi/P0yVuoATjzs/maxresdefault.jpg', '$vector': [-0.018794767558574677, 0.03362145274877548, 0.007768593728542328, -0.048206839710474014, -0.021449092775583267, 0.026007026433944702, 0.0026325411163270473, -0.017105652019381523, -0.018580276519060135, -0.01421002484858036, 0.02738781087100506, 0.036570705473423004, -0.01431727036

You can specify which **fields** you'll get back and/or whether you need the **similarity** as well:

In [15]:
documents = collection.vector_find(
    query_vector,
    limit=5,
    fields=["title", "$vector"],  # remember the dollar sign (reserved name)
    include_similarity=False,
)
for document in documents:
    print(f"\n{document}")


{'_id': '07b43be5-fa7b-4b85-abdc-821dbba4f96d', 'title': 'Predictive Coding Models of Perception', '$vector': [-0.018794767558574677, 0.03362145274877548, 0.007768593728542328, -0.048206839710474014, -0.021449092775583267, 0.026007026433944702, 0.0026325411163270473, -0.017105652019381523, -0.018580276519060135, -0.01421002484858036, 0.02738781087100506, 0.036570705473423004, -0.014317270368337631, -0.013613471761345863, -0.0016011412953957915, 0.014558572322130203, 0.024800514802336693, 0.021435687318444252, -0.009906800463795662, -0.01938461884856224, -0.013419088907539845, -0.013519631698727608, -0.012353337369859219, -0.0096520921215415, 0.010483244433999062, -0.0007373125990852714, -0.004547878168523312, -0.032709866762161255, -0.03490839898586273, -0.0035491546150296926, -0.007614428177475929, 0.004283115733414888, -0.005285190884023905, -0.02447877824306488, 0.010811683721840382, -0.003495532087981701, -0.018352380022406578, -0.00710501242429018, 0.01941142976284027, -0.0104698

You can compound with other `filter` clauses, effectively implementing **metadata filtering** on your vector searches:

In [16]:
documents = collection.vector_find(
    query_vector,
    limit=5,
    filter={"title": "Predictive Coding Models of Perception"},
)
for document in documents:
    print(f"\n{document}")


{'_id': '07b43be5-fa7b-4b85-abdc-821dbba4f96d', 'video_id': 'P0yVuoATjzs', 'text': " feedback synapses specifically and ask how the information that they contained is different than the the cell bodies in layer 2 3 so these are all experiments that are ongoing but I don't have a good answer for you but that's the right kind of question and that's the right kind of question we can ask you know when we have these kinds of models they can guide our experiments ok so this is my", 'start_second': 2196, 'end_second': 2214, 'url': 'https://www.youtube.com/watch?v=P0yVuoATjzs&t=2196s', 'title': 'Predictive Coding Models of Perception', 'thumbnail': 'https://i.ytimg.com/vi/P0yVuoATjzs/maxresdefault.jpg', '$vector': [-0.018794767558574677, 0.03362145274877548, 0.007768593728542328, -0.048206839710474014, -0.021449092775583267, 0.026007026433944702, 0.0026325411163270473, -0.017105652019381523, -0.018580276519060135, -0.01421002484858036, 0.02738781087100506, 0.036570705473423004, -0.01431727036

These options are supported for the `vector_find_one` method as well:

In [17]:
fields = ["description"]

document = collection.vector_find_one(
    query_vector,
    fields=["description"],
    include_similarity=True,  # not really necessary since True is the default
)
print(document)

{'_id': '07b43be5-fa7b-4b85-abdc-821dbba4f96d', '$similarity': 0.99999964}


## Delete a collection

In [19]:
response = astra_db.delete_collection(ASTRA_COLLECTION_NAME)
print(response)

{'status': {'ok': 1}}
